In [1]:
from pyspark import SparkContext

In [3]:
sc.stop()

In [4]:
sc = SparkContext(appName='1_MSD_Regression')

In [5]:
# Load and check the data 
import os.path
baseDir = os.path.join('data')
inputPath = os.path.join('YearPredictionMSD.csv')
fileName = os.path.join(baseDir, inputPath)
numPartitions = 2
rawData = sc.textFile(fileName, numPartitions)

In [ ]:
#Sample data
numPoints = rawData.count()
print numPoints
samplePoints = rawData.take(5)
print samplePoints

In [5]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors, Matrices
import numpy as np

In [6]:
def parsePoint(line):
    values = line.split(',')
    return LabeledPoint(values[0], Vectors.dense(values[1:]))

In [24]:
parsedSamplePoints = rawData.map(parsePoint).cache()
print parsedSamplePoints.take(2)
firstPointFeatures = parsedSamplePoints.take(1)[0].features
firstPointLabel = parsedSamplePoints.take(1)[0].label
print firstPointFeatures, firstPointLabel

[LabeledPoint(2001.0, [49.94357,21.47114,73.0775,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,-2.46783,3.32136,-2.31521,10.20556,611.10913,951.0896,698.11428,408.98485,383.70912,326.51512,238.11327,251.42414,187.17351,100.42652,179.19498,-8.41558,-317.87038,95.86266,48.10259,-95.66303,-18.06215,1.96984,34.42438,11.7267,1.3679,7.79444,-0.36994,-133.67852,-83.26165,-37.29765,73.04667,-37.36684,-3.13853,-24.21531,-13.23066,15.93809,-18.60478,82.15479,240.5798,-10.29407,31.58431,-25.38187,-3.90772,13.29258,41.5506,-7.26272,-21.00863,105.50848,64.29856,26.08481,-44.5911,-8.30657,7.93706,-10.7366,-95.44766,-82.03307,-35.59194,4.69525,70.95626,28.09139,6.02015,-37.13767,-41.1245,-8.40816,7.19877,-8.60176,-5.90857,-12.32437,14.68734,-54.32125,40.14786,13.0162,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327]), LabeledPoint(2001.0, [48.73215,18.4293,70.32679,12.94636,-10.32437,-24.83777,8.7663,-0.92019,18.76548,4.5921,2.2192,0.34006,44.38997,2056.9383

In [8]:
from pyspark.mllib.stat import Statistics,MultivariateStatisticalSummary

In [9]:
rdd = parsedSamplePoints.map(lambda a:a.features)

In [10]:
labeld = parsedSamplePoints.map(lambda a:a.label)

In [11]:
labeld.take(2)

[2001.0, 2001.0]

In [12]:
stats = Statistics.colStats(rdd)

In [13]:
print stats.count()

515345


In [19]:
print stats.mean()

[  4.33871256e+01   1.28955420e+00   8.65834709e+00   1.16412447e+00
  -6.55360070e+00  -9.52197520e+00  -2.39108942e+00  -1.79323551e+00
   3.72787581e+00   1.88238491e+00  -1.46527016e-01   2.54606336e+00
   3.37140056e+01   2.43935938e+03   1.96773420e+03   1.51485990e+03
   9.10981344e+02   8.79146721e+02   6.03737434e+02   5.17579339e+02
   3.93962215e+02   3.25733170e+02   2.88885085e+02   2.91973238e+02
   4.30319687e+01   4.33148725e+01  -4.64490057e+01  -2.76727830e+01
   1.49584906e+01   4.45148595e+01   5.13178640e+00   2.40343086e+01
   9.49881627e+00  -4.17885579e+00   4.99474661e-01   7.26522672e+01
  -5.14412560e+01   1.17921056e+02  -1.89880885e+02   2.30960821e+01
  -1.28300492e+00   1.81479733e+01  -5.19590221e+01   3.23268575e+00
  -1.48832067e+00   6.33408380e+00   7.87024269e+01   1.42696868e+02
  -8.65164764e+01   2.52407528e+01   6.37851064e+00   2.82940822e+01
   1.27722407e+01   1.70047434e+00  -1.02051553e+01   6.41012625e+01
   1.04822085e+02  -2.64808137e-02

In [20]:
print stats.numNonzeros()

[ 515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515343.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515344.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515344.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.  515345.  515345.  515345.  515345.  515345.  515345.
  515345.  515345.]


In [21]:
print stats.variance()

[  3.68152638e+01   2.66053259e+03   1.24387308e+03   2.66433469e+02
   5.22615510e+02   1.65321773e+02   2.12339488e+02   6.34225482e+01
   1.11996937e+02   4.26439263e+01   1.91043134e+01   6.92255624e+01
   4.95491232e+02   3.06028731e+06   1.59134388e+06   1.19427969e+06
   2.26298295e+05   3.32774375e+05   1.00805798e+05   9.57065390e+04
   4.58017789e+04   2.74562854e+04   3.49543141e+04   2.35547748e+04
   1.47451230e+04   5.12311157e+05   2.96894068e+05   4.76847915e+04
   2.66393937e+04   1.81947273e+04   9.81845540e+03   5.15666667e+03
   5.53709808e+03   2.86407320e+03   1.80543373e+03   1.16451354e+04
   1.72257455e+05   2.05570862e+05   6.79360087e+04   4.23287785e+04
   1.43599185e+04   1.43630035e+04   5.42026025e+03   1.47255448e+03
   1.72600967e+03   3.02218405e+03   2.21461908e+05   6.88380495e+04
   4.39041429e+04   1.49215056e+04   8.75266382e+03   5.63239395e+03
   4.89745921e+03   6.93533519e+03   3.34160201e+03   7.49813292e+04
   9.68051738e+04   7.11683813e+04

In [14]:
from pyspark.mllib.feature import VectorTransformer
from pyspark.mllib.feature import Normalizer

In [15]:
nor = Normalizer(1)
newrdd=nor.transform(rdd)

In [25]:
newrdd.take(2)

[DenseVector([0.0069, 0.003, 0.0101, 0.0012, -0.0024, -0.0018, -0.0035, -0.0017, 0.0011, -0.0003, 0.0005, -0.0003, 0.0014, 0.0846, 0.1316, 0.0966, 0.0566, 0.0531, 0.0452, 0.0329, 0.0348, 0.0259, 0.0139, 0.0248, -0.0012, -0.044, 0.0133, 0.0067, -0.0132, -0.0025, 0.0003, 0.0048, 0.0016, 0.0002, 0.0011, -0.0001, -0.0185, -0.0115, -0.0052, 0.0101, -0.0052, -0.0004, -0.0034, -0.0018, 0.0022, -0.0026, 0.0114, 0.0333, -0.0014, 0.0044, -0.0035, -0.0005, 0.0018, 0.0057, -0.001, -0.0029, 0.0146, 0.0089, 0.0036, -0.0062, -0.0011, 0.0011, -0.0015, -0.0132, -0.0114, -0.0049, 0.0006, 0.0098, 0.0039, 0.0008, -0.0051, -0.0057, -0.0012, 0.001, -0.0012, -0.0008, -0.0017, 0.002, -0.0075, 0.0056, 0.0018, -0.0075, 0.0082, 0.0021, 0.0002, -0.0032, 0.0095, -0.0003, -0.0038, 0.0003]),
 DenseVector([0.0044, 0.0017, 0.0064, 0.0012, -0.0009, -0.0023, 0.0008, -0.0001, 0.0017, 0.0004, 0.0002, 0.0, 0.004, 0.1875, 0.0552, 0.0417, 0.0708, 0.0379, 0.068, 0.0334, 0.029, 0.0249, 0.0129, 0.0289, 0.0018, -0.0059, 0.0148, 

In [26]:
nor1 = Normalizer(p=2)
newrdd1=nor.transform(rdd)
newrdd1.take(2)

[DenseVector([0.0069, 0.003, 0.0101, 0.0012, -0.0024, -0.0018, -0.0035, -0.0017, 0.0011, -0.0003, 0.0005, -0.0003, 0.0014, 0.0846, 0.1316, 0.0966, 0.0566, 0.0531, 0.0452, 0.0329, 0.0348, 0.0259, 0.0139, 0.0248, -0.0012, -0.044, 0.0133, 0.0067, -0.0132, -0.0025, 0.0003, 0.0048, 0.0016, 0.0002, 0.0011, -0.0001, -0.0185, -0.0115, -0.0052, 0.0101, -0.0052, -0.0004, -0.0034, -0.0018, 0.0022, -0.0026, 0.0114, 0.0333, -0.0014, 0.0044, -0.0035, -0.0005, 0.0018, 0.0057, -0.001, -0.0029, 0.0146, 0.0089, 0.0036, -0.0062, -0.0011, 0.0011, -0.0015, -0.0132, -0.0114, -0.0049, 0.0006, 0.0098, 0.0039, 0.0008, -0.0051, -0.0057, -0.0012, 0.001, -0.0012, -0.0008, -0.0017, 0.002, -0.0075, 0.0056, 0.0018, -0.0075, 0.0082, 0.0021, 0.0002, -0.0032, 0.0095, -0.0003, -0.0038, 0.0003]),
 DenseVector([0.0044, 0.0017, 0.0064, 0.0012, -0.0009, -0.0023, 0.0008, -0.0001, 0.0017, 0.0004, 0.0002, 0.0, 0.004, 0.1875, 0.0552, 0.0417, 0.0708, 0.0379, 0.068, 0.0334, 0.029, 0.0249, 0.0129, 0.0289, 0.0018, -0.0059, 0.0148, 

In [17]:
nor2 = Normalizer(float("inf"))
newrdd1=nor.transform(rdd)
newrdd1.take(2)

[DenseVector([0.0069, 0.003, 0.0101, 0.0012, -0.0024, -0.0018, -0.0035, -0.0017, 0.0011, -0.0003, 0.0005, -0.0003, 0.0014, 0.0846, 0.1316, 0.0966, 0.0566, 0.0531, 0.0452, 0.0329, 0.0348, 0.0259, 0.0139, 0.0248, -0.0012, -0.044, 0.0133, 0.0067, -0.0132, -0.0025, 0.0003, 0.0048, 0.0016, 0.0002, 0.0011, -0.0001, -0.0185, -0.0115, -0.0052, 0.0101, -0.0052, -0.0004, -0.0034, -0.0018, 0.0022, -0.0026, 0.0114, 0.0333, -0.0014, 0.0044, -0.0035, -0.0005, 0.0018, 0.0057, -0.001, -0.0029, 0.0146, 0.0089, 0.0036, -0.0062, -0.0011, 0.0011, -0.0015, -0.0132, -0.0114, -0.0049, 0.0006, 0.0098, 0.0039, 0.0008, -0.0051, -0.0057, -0.0012, 0.001, -0.0012, -0.0008, -0.0017, 0.002, -0.0075, 0.0056, 0.0018, -0.0075, 0.0082, 0.0021, 0.0002, -0.0032, 0.0095, -0.0003, -0.0038, 0.0003]),
 DenseVector([0.0044, 0.0017, 0.0064, 0.0012, -0.0009, -0.0023, 0.0008, -0.0001, 0.0017, 0.0004, 0.0002, 0.0, 0.004, 0.1875, 0.0552, 0.0417, 0.0708, 0.0379, 0.068, 0.0334, 0.029, 0.0249, 0.0129, 0.0289, 0.0018, -0.0059, 0.0148, 

In [17]:
parsedDataInit = rawData.map(parsePoint)
onlyLabels = parsedDataInit.map(lambda a: a.label)
minYear = onlyLabels.min()
maxYear = onlyLabels.max()
print maxYear, minYear

2011.0 1922.0


In [27]:
NewData = labeld.zip(newrdd)

In [28]:
NewData.first()

(2001.0,
 DenseVector([0.0069, 0.003, 0.0101, 0.0012, -0.0024, -0.0018, -0.0035, -0.0017, 0.0011, -0.0003, 0.0005, -0.0003, 0.0014, 0.0846, 0.1316, 0.0966, 0.0566, 0.0531, 0.0452, 0.0329, 0.0348, 0.0259, 0.0139, 0.0248, -0.0012, -0.044, 0.0133, 0.0067, -0.0132, -0.0025, 0.0003, 0.0048, 0.0016, 0.0002, 0.0011, -0.0001, -0.0185, -0.0115, -0.0052, 0.0101, -0.0052, -0.0004, -0.0034, -0.0018, 0.0022, -0.0026, 0.0114, 0.0333, -0.0014, 0.0044, -0.0035, -0.0005, 0.0018, 0.0057, -0.001, -0.0029, 0.0146, 0.0089, 0.0036, -0.0062, -0.0011, 0.0011, -0.0015, -0.0132, -0.0114, -0.0049, 0.0006, 0.0098, 0.0039, 0.0008, -0.0051, -0.0057, -0.0012, 0.001, -0.0012, -0.0008, -0.0017, 0.002, -0.0075, 0.0056, 0.0018, -0.0075, 0.0082, 0.0021, 0.0002, -0.0032, 0.0095, -0.0003, -0.0038, 0.0003]))

In [45]:
#NewData.map(lambda a:a[1:]).count()

515345

In [29]:
#Shifting labels
parsedData = NewData.map(lambda a: LabeledPoint((a[0] - 1922),a[1:]))
print type(parsedData.take(1)[0])
# View the first point
print '\n{0}'.format(parsedData.take(1))

<class 'pyspark.mllib.regression.LabeledPoint'>

[LabeledPoint(79.0, [0.00691019872744,0.00297074967418,0.0101110122385,0.00121045879758,-0.00240833512513,-0.00181238623191,-0.00346066628346,-0.00169249994839,0.0010834829411,-0.000341449274162,0.000459543794033,-0.000320332751458,0.00141204258576,0.0845531373198,0.131592878595,0.0965911809921,0.0565871961098,0.053090042877,0.0451766737282,0.0329453824655,0.0347870761396,0.0258973507623,0.0138950261406,0.024793440332,-0.00116438072422,-0.0439805863971,0.0132635698878,0.00665548049938,-0.0132359490555,-0.00249908138214,0.000272547314124,0.00476296161586,0.00162250771054,0.000189262818803,0.00107843971444,-5.11849456743e-05,-0.0184957771098,-0.0115200925339,-0.00516050762023,0.0101067465958,-0.00517008075747,-0.00043424741187,-0.00335043338605,-0.00183059580833,0.00220519616911,-0.00257415973829,0.0113669472429,0.0332866518715,-0.00142428884067,0.00437000916773,-0.00351183877672,-0.000540672638564,0.00183916306745,0.00574894632571,-0.0010

In [22]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [49]:
oldData = (parsedDataInit
           .map(lambda lp: (lp.label, 1))
           .reduceByKey(lambda x, y: x + y)
           .collect())
x, y = zip(*oldData)

# generate layout and plot data
fig, ax = preparePlot(np.arange(1920, 2050, 20), np.arange(0, 150, 20))
plt.scatter(x, y, s=14**2, c='#d6ebf2', edgecolors='#8cbfd0', alpha=0.75)
ax.set_xlabel('Year'), ax.set_ylabel('Count')
pass

In [50]:
# get data for plot
newData = (parsedData
           .map(lambda lp: (lp.label, 1))
           .reduceByKey(lambda x, y: x + y)
           .collect())
x, y = zip(*newData)

# generate layout and plot data
fig, ax = preparePlot(np.arange(0, 120, 20), np.arange(0, 120, 20))
plt.scatter(x, y, s=14**2, c='#d6ebf2', edgecolors='#8cbfd0', alpha=0.75)
ax.set_xlabel('Year (shifted)'), ax.set_ylabel('Count')
pass

In [30]:
weights = [.6, .2, .2]
seed = 42
parsedTrainData, parsedValData, parsedTestData = parsedData.randomSplit(weights,seed)
parsedTrainData.cache()
parsedValData.cache()
parsedTestData.cache()
nTrain = parsedTrainData.count()
nVal = parsedValData.count()
nTest = parsedTestData.count()

print nTrain, nVal, nTest, nTrain + nVal + nTest
print parsedData.count()

308870 103451 103024 515345
515345


In [24]:
#Create and evaluate a baseline model 
averageTrainYear = (parsedTrainData.map(lambda x:x.label).mean())
print averageTrainYear

76.3946391283


In [22]:
def squaredError(label, prediction):
    return (float(label) - float(prediction))**2 
def calcRMSE(labelsAndPreds):
    return np.sqrt(labelsAndPreds.map(lambda (label,prediction) : squaredError(label,prediction)).reduce(lambda x,y : x+y)/labelsAndPreds.count())

In [23]:
labelsAndPredsTrain = parsedTrainData.map(lambda x:(x.label,76.39463))
#labelsAndPredsTrain.take(10)                                          
rmseTrainBase = calcRMSE(labelsAndPredsTrain)

labelsAndPredsVal = parsedValData.map(lambda x:(x.label,76.39463))
rmseValBase = calcRMSE(labelsAndPredsVal)

labelsAndPredsTest = parsedTestData.map(lambda x:(x.label,76.39463))
rmseTestBase = calcRMSE(labelsAndPredsTest)

print 'Baseline Train RMSE = {0:.3f}'.format(rmseTrainBase)
print 'Baseline Validation RMSE = {0:.3f}'.format(rmseValBase)
print 'Baseline Test RMSE = {0:.3f}'.format(rmseTestBase)

Baseline Train RMSE = 10.939
Baseline Validation RMSE = 10.947
Baseline Test RMSE = 10.891


In [31]:
#LinearRegressionWithSGD
from pyspark.mllib.regression import LinearRegressionWithSGD
# Values to use when training the linear regression model

numIters = 500  # iterations
alpha = 1.0  # step
miniBatchFrac = 1.0  # miniBatchFraction
reg = 1e-1  # regParam
regType = 'l2'  # regType
useIntercept = True  # intercept

In [32]:

firstModel = LinearRegressionWithSGD.train(parsedTrainData, iterations = numIters, step = alpha, miniBatchFraction = miniBatchFrac,
                                          regParam = reg, regType = regType, intercept = useIntercept)


weightsLR1 = firstModel.weights
interceptLR1 = firstModel.intercept
print weightsLR1, interceptLR1

[0.228414391638,0.0546818869785,0.0370837177161,-0.0106080387818,-0.0506825241564,-0.0781862507568,0.00111773420804,-0.0105136373253,0.0210990889886,0.0123401382994,0.000292223913911,0.00503419457498,0.137697884856,9.37415875291,7.02356712377,5.66038119902,3.69072090788,3.37369982082,2.38126378021,2.14534511673,1.51664819481,1.33707934548,1.15163829981,1.15211284798,0.215125328166,-0.218598923232,-0.235161038259,-0.0914619550957,0.100241576124,0.162370554724,-0.0359845141191,0.107381653422,0.012161271736,-0.0238703101086,0.0132338296163,0.202545602652,-0.46580160727,0.549765768769,-0.697828981887,-0.0454199605641,-0.0509477289537,0.123110656587,-0.228885380141,0.0178483537758,-0.00951924852073,-0.017728123699,0.602605450004,0.441862186858,-0.368621417638,0.13241398712,0.0262170877982,0.108274949648,0.0715944499905,-0.0195404136492,-0.01826362135,0.116429710834,0.226022058778,-0.0285013160251,0.0251327113937,-0.189510852508,-0.015813730267,0.00407926397477,-0.0539270804825,-0.5009979295

In [ ]:
#parsedData.saveAsTextFile("parsedData")

In [33]:
#validation
labelsAndPreds = parsedValData.map(lambda rec: (rec.label, firstModel.predict(rec.features)))
rmseValLR1 = calcRMSE(labelsAndPreds)

print ('Validation RMSE:\n\tBaseline = {0:.3f}\n\tLR1 = {1:.3f}').format(rmseValBase, rmseValLR1)

Validation RMSE:
	Baseline = 10.947
	LR1 = 12.810


In [34]:
from pyspark.mllib.regression import LassoWithSGD 

In [ ]:
#LassoWithSGD
Model2 = LassoWithSGD.train(parsedTrainData,iterations = numIters,step = alpha,regParam = reg,miniBatchFraction = miniBatchFrac)

In [ ]:
#validation_LassoSGD
labelsAndPreds1 = parsedValData.map(lambda rec: (rec.label,Model2.predict(rec.features)))
rmseValLA1 = calcRMSE(labelsAndPreds1)

print ('Validation RMSE:\n\tBaseline = {0:.3f}\n\tLR1 = {1:.3f}\n\tLA1= {2:3f}').format(rmseValBase,rmseValLR1, rmseValLA1)

In [ ]:
#RidgewithSGD
from pyspark.mllib.regression import RidgeRegressionWithSGD
Model3 = RidgeRegressionWithSGD.train(parsedTrainData,iterations = numIters,step = alpha,regParam = reg,miniBatchFraction = miniBatchFrac)
#validation_LassoSGD
labelsAndPreds2 = parsedValData.map(lambda rec: (rec.label,Model3.predict(rec.features)))
rmseValRD1 = calcRMSE(labelsAndPreds2)

print ('Validation RMSE:\n\tBaseline = {0:.3f}\n\tLR1 = {1:.3f}\n\tLA1= {2:3f}\n\tRD1= {3:3f}').format(rmseValBase,rmseValLR1, rmseValLA1,rmseValRD1)

In [ ]:
# Evaluating the model on training data
labelsAndPreds = parsedTrainData.map(lambda p: (p.label, float(model.predict(p.features))))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedTrainData.count())
print("Training Error = " + str(trainErr))

In [ ]:
#Grid search
bestRMSE = rmseValLR1
bestRegParam = reg
bestModel = firstModel

numIters = 500
alpha = 1.0
miniBatchFrac = 1.0
for reg in [1e-10, 1e-5, 1]:
    model = LinearRegressionWithSGD.train(parsedTrainData, numIters, alpha,
                                          miniBatchFrac, regParam=reg,
                                          regType='l2', intercept=True)
    labelsAndPreds = parsedValData.map(lambda lp: (lp.label, model.predict(lp.features)))
    rmseValGrid = calcRMSE(labelsAndPreds)
    print rmseValGrid

    if rmseValGrid < bestRMSE:
        bestRMSE = rmseValGrid
        bestRegParam = reg
        bestModel = model
rmseValLRGrid = bestRMSE

print ('Validation RMSE:\n\tBaseline = {0:.3f}\n\tLR1 = {1:.3f}\n\tLA1= {2:3f}\n\tRD1= {3:3f}\n\tLRGrid= {4:3f}').format(rmseValBase,rmseValLR1, rmseValLA1,rmseValRD1,rmseValGrid)